<a href="https://colab.research.google.com/github/Netdrum/MARIA/blob/main/HF_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
import datasets
import json
import os

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# Define the dataset name and path
dataset_name = "IRCG_VHF"  # Replace with your desired dataset name
dataset_path = "//content/drive/MyDrive/Dataset"  # Replace with the path to your dataset

def load_dataset(dataset_path):
  """Loads the dataset from the specified path.

  Args:
    dataset_path: Path to the dataset directory.

  Returns:
    A Hugging Face dataset.
  """

In [ ]:
from datasets import load_dataset, Audio, Features, Value
import json
import os

# Define the data directory
data_dir = "/content/drive/MyDrive/Dataset"

# Define the dataset features
features = Features({
    "file_path": Value("string"),
    "duration": Value("float32"),
    "sample_rate": Value("int32"),
    "length": Value("int32"),
    "frame_rate": Value("int32"),
    "audio": Audio(sampling_rate=16000),
    "transcription": Value("string")
})

# Load the dataset, extracting metadata from JSON files
def extract_metadata(example):
    # Assuming JSON files are named similarly to audio files with a .json extension:
    json_file_path = example["file_path"]  # Get the JSON file path

    # Prepend the data directory to the JSON file path
    json_file_path = os.path.join(data_dir, json_file_path)

    # Add .json extension if it's missing
    if not json_file_path.lower().endswith(".json"):
        json_file_path += ".json"

    # Check if the JSON file exists
    if not os.path.exists(json_file_path):
        print(f"Warning: JSON file not found: {json_file_path}")
        return example  # Skip this example if JSON file is not found

    try:
        # Open the JSON file with error handling and encoding specified
        with open(json_file_path, 'r', encoding='latin-1') as f:
            metadata = json.load(f)
            example["duration"] = metadata["duration"]
            example["sample_rate"] = metadata["sample_rate"]
            example["length"] = metadata["length"]
            example["frame_rate"] = metadata["frame_rate"]
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file: {json_file_path}")
        print(f"Error message: {e}")
        return example  # Skip this example if JSON decoding fails

    # Construct the audio file path based on the JSON file path
    audio_file_path = json_file_path.replace(".json", ".wav")  # Change .json to .wav

    # Update the "audio" field with the path to the audio file
    example["audio"] = {"path": audio_file_path}

    return example

dataset = load_dataset("json", data_files={"train": f"{data_dir}/*.json"}, features=features)
dataset = dataset.map(extract_metadata)

# Push the dataset to the Hugging Face Hub
dataset.push_to_hub("IRCG_VHF")

In [6]:
# in this cell added in code to split the dataset into train and test splite and push to hf

from datasets import load_dataset, Audio, Features, Value
import json
import os

# Define the data directory
data_dir = "/content/drive/MyDrive/Dataset"

# Define the dataset features
features = Features({
    "file_path": Value("string"),
    "duration": Value("float32"),
    "sample_rate": Value("int32"),
    "length": Value("int32"),
    "frame_rate": Value("int32"),
    "audio": Audio(sampling_rate=16000),
    "transcription": Value("string")
})

# Load the dataset, extracting metadata from JSON files
def extract_metadata(example):
    # Assuming JSON files are named similarly to audio files with a .json extension:
    json_file_path = example["file_path"]  # Get the JSON file path

    # Prepend the data directory to the JSON file path
    json_file_path = os.path.join(data_dir, json_file_path)

    # Add .json extension if it's missing
    if not json_file_path.lower().endswith(".json"):
        json_file_path += ".json"

    # Check if the JSON file exists
    if not os.path.exists(json_file_path):
        print(f"Warning: JSON file not found: {json_file_path}")
        return example  # Skip this example if JSON file is not found

    try:
        # Open the JSON file with error handling and encoding specified
        with open(json_file_path, 'r', encoding='latin-1') as f:
            metadata = json.load(f)
            example["duration"] = metadata["duration"]
            example["sample_rate"] = metadata["sample_rate"]
            example["length"] = metadata["length"]
            example["frame_rate"] = metadata["frame_rate"]
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file: {json_file_path}")
        print(f"Error message: {e}")
        return example  # Skip this example if JSON decoding fails

    # Construct the audio file path based on the JSON file path
    audio_file_path = json_file_path.replace(".json", ".wav")  # Change .json to .wav

    # Update the "audio" field with the path to the audio file
    example["audio"] = {"path": audio_file_path}

    return example

dataset = load_dataset("json", data_files={"train": f"{data_dir}/*.json"}, features=features)
dataset = dataset.map(extract_metadata)

# Split the dataset into train and test
train_testvalid_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

# Extract train and test datasets
train_dataset = train_testvalid_dataset["train"]
test_dataset = train_testvalid_dataset["test"]

# Push the datasets to the Hugging Face Hub
train_dataset.push_to_hub("IRCG_VHF", split="train")
test_dataset.push_to_hub("IRCG_VHF", split="test")

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/610 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Netdrum/IRCG_VHF/commit/4083f18dd421c9cb0bae6bd3a9adb965a0d7bc9a', commit_message='Upload dataset', commit_description='', oid='4083f18dd421c9cb0bae6bd3a9adb965a0d7bc9a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Netdrum/IRCG_VHF', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Netdrum/IRCG_VHF'), pr_revision=None, pr_num=None)